In [1]:
! pip install python-gdcm
! pip install pylibjpeg pylibjpeg-libjpeg pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 3.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
import os
import nibabel as nib
import SimpleITK as sitk
from tqdm import tqdm
from tensorflow import keras
from keras import layers, models
import tensorflow_io as tfio
import cv2
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import resize
from tqdm import tqdm
import tensorflow_addons as tfa

In [3]:
def load_image(path):
    img = pydicom.read_file(path)
    img = img.pixel_array
    return resize(img[:,20:-20], (100, 60))

In [4]:
def window(img):
    img[np.where(img<0)] = 0
    mean = img.mean()
    std = img.std()
    img[np.where(img>(mean+(3*std)))] = (mean+(3*std))
    img = img-img.min()
    maxx = img.max()
    img[np.where(img<(maxx/2))] = maxx/2
    img = (img-(maxx/2)) / (maxx/2)
    return img

In [5]:
model = tf.keras.models.load_model('../input/rsna-cervical-fracture-training/final_model/model')

2022-10-06 20:42:07.556238: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
patient_pred = {}
UIDs = os.listdir("../input/rsna-2022-cervical-spine-fracture-detection/test_images")
for p in UIDs:
    path = "../input/rsna-2022-cervical-spine-fracture-detection/test_images/"+p+"/"
    len10percent = len(os.listdir(path))//10
    final_ct = window(load_image(path+str(len10percent)+".dcm")).reshape(100,60,1)
    for image in range(len10percent, len(os.listdir(path))-len10percent):
        img = window(load_image(path+str(image)+".dcm")).reshape(100,60,1)
        final_ct = np.append(final_ct, img, axis=-1)
    resized_final_ct = cv2.resize(final_ct[:,0,:], (100,100)).reshape(100,100,1)
    for i in range(1,final_ct.shape[1]):
        resized_final_ct = np.append(resized_final_ct, cv2.resize(final_ct[:,i,:], (100,100)).reshape(100,100,1), axis=-1)
    resized_final_ct = resized_final_ct.reshape(1,100,100,60)
    pred = model.predict(resized_final_ct)
    for i in range(1, 9):
        if i==8:
            patient_pred[p+"_"+"patient_overall"] = pred[0][0][0]
            break
        else:
            patient_pred[p+"_"+"C"+str(i)] = pred[i][0][0]
submission  = pd.DataFrame.from_dict({"row_id": patient_pred.keys(), "fractured": patient_pred.values()})
submission.to_csv('./submission.csv', index=False)

2022-10-06 20:42:19.508303: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [7]:
submission

,row_id,fractured
0,1.2.826.0.1.3680043.22327_C1,0.017870
1,1.2.826.0.1.3680043.22327_C2,0.028587
2,1.2.826.0.1.3680043.22327_C3,0.016075
3,1.2.826.0.1.3680043.22327_C4,0.025984
4,1.2.826.0.1.3680043.22327_C5,0.041403
5,1.2.826.0.1.3680043.22327_C6,0.064498
6,1.2.826.0.1.3680043.22327_C7,0.215468
7,1.2.826.0.1.3680043.22327_patient_overall,0.237880
8,1.2.826.0.1.3680043.25399_C1,0.123502
9,1.2.826.0.1.3680043.25399_C2,0.345309
